In [18]:
import gym
import rec_gym
from rec_gym.wrappers import StatsWrapper, FlattenObservationsWrapper
from copy import deepcopy
from rec_gym.runner import run_experiment

from agents.baselines import RandomAgent, DotProdAgent, MFAgent
from agents.dqn import Qagent
from agents.ddpg import DDPGAgent
import tensorflow as tf

import tqdm

from agents import PopularityAgent, SVDAgent, LinUCB, HLinUCB, create_drr_agent, RandomAgent

from rec_gym.wrappers import StatsWrapper, DRR_EmbBaselinesWrapper, ExplicitUserItemWrapper, DRR_ExplicitUserItemWrapper, DRR_BaselinesWrapper

from rec_gym.envs import MovieLensDRR, MovieLens100

import time
import numpy as np

In [19]:
def train_eval(agent, env, train_steps, eval_steps):

    done = True
    r = None
    info = None

    ts = []
    for _ in tqdm.tqdm_notebook(range(train_steps)):
        
        start = time.time()
        
        if done:
            if r:
                agent.end_episode(r, info)
            obs = env.reset()
            a = agent.begin_episode(obs)
    #         print(a)
            obs, r, done, info = env.step(a)
    #         done = False
        else:
            a = agent.step(r, obs, info)
    #         print(a)
            obs, r, done, info = env.step(a)
        
        end = time.time()
        ts.append(end-start)
    
    mean_train_t = np.mean(ts)
    
    done = True
    r = None
    info = None
    agent.eval_mode = True
    
    ts = []
    for _ in tqdm.tqdm_notebook(range(eval_steps)):
        start = time.time()
        if done:
            if r:
                agent.end_episode(r, info)
            obs = env.reset()
            a = agent.begin_episode(obs)
            obs, r, done, info = env.step(a)
        else:
            a = agent.step(r, obs, info)
            obs, r, done, info = env.step(a)
        end = time.time()
        ts.append(end-start)
    
    mean_eval_t = np.mean(ts)
    
    return mean_train_t, mean_eval_t

In [23]:
def prepare_random_agent_and_env(base_env):
    env = deepcopy(base_env)
    wrapped = FlattenObservationsWrapper(env)
    action_size = env.unwrapped.n_rec
    agent = RandomAgent(action_size = action_size)    
    return agent, wrapped, env

def prepare_ddpg_agent_and_env(base_env):
    env = deepcopy(base_env)
    wrapped = DRR_BaselinesWrapper(env)
    unwrapped = env.unwrapped

    state_dim = unwrapped.embedding_dimension
    action_dim = unwrapped.embedding_dimension
    action_size = unwrapped.n_rec


    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    agent = DDPGAgent(action_size = action_size,
                     state_dim = state_dim,
                     action_dim = action_dim,
                     gamma = .9,
                     sess = sess,
                     optimizer = tf.train.AdamOptimizer(
                         learning_rate=0.01
                     ),
                     max_tf_checkpoints_to_keep = 3,
                     experience_size = 300,
                     per = True,
                     batch_size = 64, 
                     start_steps = 400)
    sess.run(tf.global_variables_initializer())
    return agent, wrapped, env

In [24]:
def make_env(session_size=10, seed=123):
#     env = MovieLensDRR(embedding_dimension=20, 
#              n_items_to_recommend=1, 
#              env_seed=0,
#              normalize_reward=True,
# #              filename="/home/anton/Datasets/MovieLens/ml-100k/u.data",
# #              sep='\t',
#              filename="/home/anton/Datasets/MovieLens/ml-1m/ratings.dat",\
#              sep='::',\
# #              session_time = 20 * 60,
#              session_size = session_size,
#              cache_dir=CACHE_DIR,
#              shuffle_sessions=False)
    
#     env = MovieLensDRR(embedding_dimension=40, 
#              n_items_to_recommend=1, 
#              env_seed=seed,
#              normalize_reward=True,
#              filename="/home/anton/Datasets/MovieLens/ml-100k/u.data",
#              sep='\t',
#     #              session_time = 20 * 60,
#              session_size = session_size,
#              cache_dir=CACHE_DIR,
#              shuffle_sessions=True
#                       )
    env = MovieLens100(
                 n_items_to_recommend=1,
                 env_seed=seed,
                 normalize_reward=False,
                 session_time=None,
                 session_size=session_size,
                 shuffle_sessions=True,
                 )
    
    return StatsWrapper(env)

envfn = {
    'ml100k_s30_seed10' : lambda : make_env(30, 100)
}
agents = {
        'random' : lambda x : prepare_random_agent_and_env(x),
        'ddpg'   : lambda x : prepare_ddpg_agent_and_env(x),
        'mf_agent' : lambda x : prepare_mf_agent_and_env(x),
}

n_steps = [
           10000, 
           #4000, 
           #16000, 
           #16000
          ]

In [25]:
from collections import defaultdict

results = defaultdict( defaultdict) 
for envid, n_step in tqdm.tqdm_notebook(zip(envfn.keys(), n_steps), leave=False, position=0):

    base_env = StatsWrapper(envfn[envid]())
    base_env.unwrapped.n_rec

    for agent_name, prepare in agents.items():
        agent, wrapped, env = prepare(base_env)
        #run_experiment(wrapped, agent, t_train=n_step, t_test=0)
        train_eval(agent, wrapped, train_steps=1000, eval_steps=1000)
        
        results[envid][agent_name] = env.interactions

/home/anton/anaconda3/envs/datascience/lib/python3.7/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


ValueError: not enough values to unpack (expected 2, got 1)